# Transverse Wave Simulator
This script simulates a wave with parameters that the user provides using `Plots.jl` and `Interact.jl`. The user can choose between two modes based on the data avaiable to him/her. The entire UI is implemented dynamically based on the user's input. At the end of the calculation, an endlessly looping .gif (at 60fps) of the wave is saved as "wave.gif" in the location of the Jupyter Notebook, the .gif is displayed along with some other numerical values useful for further analysis of wave.

## 1.1 Modes

### 1.1.0 Classical Mode:
The "Classical Mode" simulates the wave using classic parameters such as amplitude, frequency, wavelength, time period etc. The user can choose between different modes to enter the time variant ( Angular Frequency (ω), Time period (T), or Frequency (f)) and the position variant (Angular wave number (k), or Wavelength (λ)). This offers a great amount of flexibility to the user since the user does not need to convert between different forms of measurement of the same parameter.

### 1.1.1 Mechanical Mode:
The "Mechanical Mode" simulates the wave using mechanical properties of the string carrying the transverse wave such as Tension in the string, Linear Density of the string, etc. Again, the user can choose between "Linear Density" and "Mass and Length" of the string and the same is true for "Position variant" and "Time variant". The tension and linear density of the string eliminate the need for **both** position and time variants. Using mechanical properties, we need only one of these two.

In both modes, a phase constant field is also included to allow the wave to be shifted in time with respect to a sine curve.

## 1.2 Units
All units are SI units. This means:
- Amplitude, Wavelength (λ), Length of String are all in _metres_
- Angular wave number (k) is in _radians/metre_
- Angular frequency (ω) is in _radians/second_
- Frequency (f) is in _Hertz or seconds⁻¹_
- Linear Density is in _kilograms/metre_
- Time period (T) is in _seconds_
- Phase constant is in _radians_
- Tension is in _Newtons_
- Mass is in _kilograms_
- Length is in _metres_

In [8]:
using Interact, Plots

## 1.3 Creating the UI Elements
The below code creates all the elements required for creating the UI of the app. It only initializes them. They are not assigned any default values. `hbox` and `vbox` layouts are used alongwith padding to give the UI a finished look instead of just a bunch of UI elements stacked on top of each other.

In [9]:
mode_toggle = togglebuttons(["Classical Mode", "Mechanical Mode"], value = "Classical Mode")

amplitude = spinbox(label = "Amplitude: ")

pos_var_mode = togglebuttons(["k", "λ"], label = "Position Variant k or λ: ")
pos_var_val = spinbox()
pos_var_box = hbox(pos_var_mode, pad(0.2em, pos_var_val))

time_var_mode = togglebuttons(["ω", "T", "f"], label = "Time Variant ω or T or f: ")
time_var_val = spinbox()
time_var_box = hbox(time_var_mode, pad(0.2em, time_var_val))

phase_const = spinbox(label = "Phase constant/shift: ")

tension = spinbox(label = "Tension in string carrying wave: ")

string_char_mode = togglebuttons(["Linear Density", "Mass and Length"], label = "String characteristics: ")
string_char_mode_box = hbox(string_char_mode)
string_char_linear_density = spinbox(label = "Linear Density of String: ")
string_char_mass = spinbox(label = "Mass of String: ")
string_char_length = spinbox(label = "Length of String: ")
string_char_box = vbox(string_char_mode_box, pad(1em, string_char_linear_density))


wave_char_mode = togglebuttons(["Position Variant", "Time Variant"], label = "Wave characteristics: ")
wave_char_mode_box = hbox(wave_char_mode)

pos_var_mode_mech  = togglebuttons(["k", "λ"], label = "Position Variant k or λ: ")
pos_var_val_mech = spinbox()
pos_var_box_mech = hbox(pos_var_mode, pad(0.2em, pos_var_val_mech))

time_var_mode_mech = togglebuttons(["ω", "T", "f"], label = "Time Variant ω or T or f: ")
time_var_val_mech = spinbox()
time_var_box_mech = hbox(time_var_mode, pad(0.2em, time_var_val_mech))

wave_char_box_mech = vbox(wave_char_mode_box, pad(0.2em, pos_var_box_mech))

calculate_button = button("Calculate!")

calc_box = vbox(mode_toggle, pad(1em, amplitude), pad(1em, pos_var_box), pad(1em, time_var_box), pad(1em, phase_const), pad(1em, calculate_button));

## 1.4 Switching between Classic and Mechanical Mode
The below code utilizes the `on()` function to re-render the UI when the mode is changed. This is a major part of the working of the dynamic UI. The function checks the value of the mode slider created in the previous cell to determine the mode the simulator is in. This uses the `IJulia.clear_output()` to clear the previous un-updated UI.

In [10]:
on(
    function (unused_var2)
        if observe(mode_toggle).val == "Classical Mode"
            calc_box = vbox(mode_toggle, pad(1em, amplitude), pad(1em, pos_var_box), pad(1em, time_var_box), pad(1em, phase_const), pad(1em, calculate_button))
            IJulia.clear_output(true)
            display(calc_box)
        else
            calc_box = vbox(mode_toggle, pad(1em, amplitude), pad(1em, tension), pad(1em, string_char_box), pad(1em, wave_char_box_mech), pad(1em, phase_const), pad(1em, calculate_button))
            IJulia.clear_output(true)
            display(calc_box)
        end
    end,
    mode_toggle
)

#12 (generic function with 1 method)

## 1.5 Switching between "Linear Density" and "Mass and Length" in Mechanical Mode
The below code again uses the `on()` function to update the UI to reflect the change in the choice of mechanical properties each time the toggle is updated. Since the UI is being re-updated, the choice between Position variant and Time variant needs to be updated as well. If this is not done, then when Time variant is selected and the mechanical properties is changed, then the Time variant is selected, while the UI for the Position variant is shown. Again, just as we saw before, the `IJulia.clear_output()` is used to clear the previous un-updated UI.

In [11]:
on(    
    function (unused_var1)
        if observe(string_char_mode).val == "Linear Density"
            if observe(wave_char_mode).val == "Position Variant"
                wave_char_mode_box = hbox(wave_char_mode)
                wave_char_box_mech = vbox(wave_char_mode_box, pad(1em, pos_var_box_mech))
            else
                wave_char_mode_box = hbox(wave_char_mode)
                wave_char_box_mech = vbox(wave_char_mode_box, pad(1em, time_var_box_mech))
            end
            string_char_mode_box = hbox(string_char_mode)
            string_char_box = vbox(string_char_mode_box, pad(1em, string_char_linear_density))
            calc_box = vbox(mode_toggle, pad(1em, amplitude), pad(1em, tension), pad(1em, string_char_box), pad(1em, wave_char_box_mech), pad(1em, phase_const), pad(1em, calculate_button))
            IJulia.clear_output(true)
            display(calc_box)
        else
            if observe(wave_char_mode).val == "Position Variant"
                wave_char_mode_box = hbox(wave_char_mode)
                wave_char_box_mech = vbox(wave_char_mode_box, pad(1em, pos_var_box_mech))
            else
                wave_char_mode_box = hbox(wave_char_mode)
                wave_char_box_mech = vbox(wave_char_mode_box, pad(1em, time_var_box_mech))
            end
            string_char_mode_box = hbox(string_char_mode)
            string_char_box = vbox(string_char_mode_box, pad(1em, string_char_mass), pad(1em, string_char_length))
            calc_box = vbox(mode_toggle, pad(1em, amplitude), pad(1em, tension), pad(1em, string_char_box), pad(1em, wave_char_box_mech), pad(1em, phase_const), pad(1em, calculate_button))
            IJulia.clear_output(true)
            display(calc_box)
        end
    end,
    string_char_mode
)

#14 (generic function with 1 method)

## 1.6 Switching between "Position Variant" and "Time Variant"
Similar to how we switched between "Linear Density" and "Mass and Length", we switch between Position and Time variants. In fact, the code is almost same, with only the UI element names switched up. Again, we update the "Linear Density" and "Mass and Length" toggle UI to keep the UI working coherently. For the third time, we use `IJulia.clear_output()` to clear the previous un-updated UI.

In [12]:
on(
    function (unused_var3)
        if observe(wave_char_mode).val == "Position Variant"
            if observe(string_char_mode).val == "Linear Density"    
                string_char_mode_box = hbox(string_char_mode)
                string_char_box = vbox(string_char_mode_box, pad(1em, string_char_linear_density))
            else  
                string_char_mode_box = hbox(string_char_mode)
                string_char_box = vbox(string_char_mode_box, pad(1em, string_char_mass), pad(1em, string_char_length))
            end
            wave_char_mode_box = hbox(wave_char_mode)
            wave_char_box_mech = vbox(wave_char_mode_box, pad(1em, pos_var_box_mech))
            calc_box = vbox(mode_toggle, pad(1em, amplitude), pad(1em, tension), pad(1em, string_char_box), pad(1em, wave_char_box_mech), pad(1em, phase_const), pad(1em, calculate_button))
            IJulia.clear_output(true)
            display(calc_box)
        else
            if observe(string_char_mode).val == "Linear Density"    
                string_char_mode_box = hbox(string_char_mode)
                string_char_box = vbox(string_char_mode_box, pad(1em, string_char_linear_density))
            else  
                string_char_mode_box = hbox(string_char_mode)
                string_char_box = vbox(string_char_mode_box, pad(1em, string_char_mass), pad(1em, string_char_length))
            end
            wave_char_mode_box = hbox(wave_char_mode)
            wave_char_box_mech = vbox(wave_char_mode_box, pad(1em, time_var_box_mech))
            calc_box = vbox(mode_toggle, pad(1em, amplitude), pad(1em, tension), pad(1em, string_char_box), pad(1em, wave_char_box_mech), pad(1em, phase_const), pad(1em, calculate_button))
            IJulia.clear_output(true)
            display(calc_box)
        end
    end,
    wave_char_mode
)

#16 (generic function with 1 method)

## 1.7 Taking in Values and Peforming the Simulation

### 1.7.0 Overview of methods and Packages used
Until now, we have worked on ways to create and update the UI when necessary. Now, we must take in the input for the parameters, sanitize the input, and narrow down our hundreds of combinations of parameters into a few variables that we can plug into a formula and simulate the wave using Plots and animation.

I chose `Plots.jl` for this especially since it supports a number of backends, has great documentation, is **very** flexible with input, allows animation, and has concise syntax. However, I could also have used Makie.jl, or some other plotting package, and that would work fine, but it would take time to render the PNGs of the graphs, then stitch them into an animation. 

We finally use the `on()` function to detect clicks on the "Calculate!" Button to start the simulation.

### 1.7.1 Sanitizing the Input
First, to sanitize the input, I built two functions exist() to check if a list of Observables all exist and are not "nothing", and more_than_zero() to check if a list of Observables all have values higher than zero. If the input does not follow the conditions of the parameters, an error is thrown which for some reason do not show up in the Output. Instead, no calculaton is made. 

### 1.7.2 Boiling all the cases down to one formula
If the values all follow the conditions of the parameters however, the type of value (time variant, position variant, etc.) is checked, and five final values are generated which are used in the formula.

These values are:
- Amplitude
- Angular Wave Number
- Angular Frequency
- Phase Constant
- Wave Speed (Not used in formula, but is one of the results useful for further analysis of wave)

At the end, they are plugged into the formula:

$$y(x, t) =  y_{max}sin(kx - \omega t)$$

where:
- $$y(x, t)$$ is the positon of the string at position x and time t
- $$y_{max}$$ is the amplitude
- $$k$$ is the Angular Wave Number
- $$\omega$$ is the Angular Frequency

^For some reason, this doesn't render well on Jupyter Notebook on my Laptop. I hope this renders well on Gist.

### 1.7.3 Plotting and animating the graph for the wave
Then, the function is plotted, and using the `@animate` macro, the animation is made. The graph is plotted such that:
- Exactly 2 Wavelengths are _always_ on the graph
- The animation is _always_ at 60fps, adjusting the speed of the wave to be accurate
- The .gif is saved in the same directory as the Jupyter Notebook

In [13]:
function exist(thingsa)
    for itema in thingsa
        if observe(itema).val != nothing
            continue
        else
            return false
        end
    end
    return true
end

function more_than_zero(thingsb)
    if exist(thingsb)
        for itemb in thingsb
            if observe(itemb).val > 0
                continue
            else
                return false
            end
        end
        return true
    else 
        return false
    end
end    

on(
    function(unused_var4)
        if observe(mode_toggle).val == "Classical Mode"
            if !(exist([amplitude, time_var_val, pos_var_val, phase_const]))
                value_error = alert("Please enter valid values")
                value_error()
                return
            elseif !(more_than_zero([amplitude, time_var_val, pos_var_val]))
                value_error = alert("Please enter non-zero and positive values") 
                value_error()
                return
            else
                final_amplitude = observe(amplitude).val
                if observe(pos_var_mode).val == "k"
                    final_k = observe(pos_var_val).val
                else
                    final_k = 2pi/observe(pos_var_val).val
                end
                
                if observe(time_var_mode).val == "ω"
                    final_angular_freq = observe(time_var_val).val
                elseif observe(time_var_mode).val == "T"
                    final_angular_freq = 2pi/observe(time_var_val).val
                else
                    final_angular_freq = 2pi*observe(time_var_val).val
                end
                
                final_phase_const = observe(phase_const).val
                final_wave_speed = final_angular_freq/final_k
            end
        
        else 
            value_error = alert("Please enter valid values")
            if (!(more_than_zero([amplitude, tension, phase_const])))
                value_error()
                return
            
            elseif (observe(string_char_mode).val == "Linear Density") && (!(more_than_zero([string_char_linear_density])))
                value_error()
                return
            elseif (observe(string_char_mode).val == "Mass and Length") && (!(more_than_zero([string_char_mass, string_char_length])))
                value_error()
                return
            elseif (observe(wave_char_mode).val == "Position Variant") && (!(more_than_zero([pos_var_val_mech])))
                value_error()
                return
            elseif (observe(string_char_mode).val == "Time Variant") && (!(more_than_zero([time_var_val_mech])))
                value_error()
                return
            else                
                final_amplitude = observe(amplitude).val
                final_tension = observe(tension).val
                final_phase_const = observe(phase_const).val
                
                if observe(string_char_mode).val == "Linear Density"
                    d = observe(string_char_linear_density).val
                    final_wave_speed = sqrt(final_tension/d)
                else
                    d = (observe(string_char_mass).val)/(observe(string_char_length).val)
                    final_wave_speed = sqrt(final_tension/d)
                end
                
                if observe(wave_char_mode).val == "Position Variant"
                    if observe(pos_var_mode).val == "k"
                        final_k = observe(pos_var_val_mech).val
                        final_angular_freq = final_k * final_wave_speed
                    else
                        final_k = 2pi / observe(pos_var_val_mech).val
                        final_angular_freq = final_k * final_wave_speed
                    end
                else
                    if observe(time_var_mode).val == "ω"
                        final_angular_freq = observe(time_var_val_mech).val
                        final_k = final_angular_freq / final_wave_speed
                    elseif observe(time_var_mode).val == "T"
                        final_angular_freq = 2pi / observe(time_var_val_mech).val
                        final_k = final_angular_freq / final_wave_speed
                    else
                        final_angular_freq = 2pi * observe(time_var_val_mech).val
                        final_k = final_angular_freq / final_wave_speed
                    end
                end
            end
        end
        tmax = 2pi / final_angular_freq
        xvals = range(0, stop = 2pi / final_k, length = 1000)

        anim = @animate for dt in range(0, stop = tmax, length = Int(round(tmax * 60)))
            f(x) = final_amplitude * sin(final_k * x - final_angular_freq * dt + final_phase_const)
            plot(xvals, f)
        end
        
        myanim = gif(anim, "wave.gif", fps = 60)
        
        everything = vbox(calc_box, myanim)
        IJulia.clear_output(true)
        display(everything)
        results = """The speed of the wave is: $final_wave_speed m/s
            The wavelength of the wave is: $(2pi/final_k) m
            The frquency of this wave is: $(final_angular_freq/2pi) Hz
            The amplitude of this wave is: $final_amplitude m
            The phase constant of this wave is: $final_phase_const radians"""
        println(results)
    end,
        calculate_button
)


#18 (generic function with 1 method)

## 1.8 Running the Simulation
To run the simulation, we simply type `display(calc_box)`. This shows the UI from where we can control and run the simulation. The calculation takes some seconds, and the gif is shown along with the useful results.

In [14]:
display(calc_box)

(div { style=Dict("display" => "flex","flex-direction" => "column") }
  (div { style=Dict("display" => "flex","flex-direction" => "column") }
    (div { className="interact-widget" }
            (div { attributes="data-bind" => "foreach : options_js" className="buttons has-addons is-centered" }
        (span { attributes=Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}") }
          (label { attributes=Dict("data-bind" => "text : key") }))))
    (div { style=Dict("padding" => 1.0em) }
      Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Amplitude: "], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:attributes => Dict{Any,Any}(:type => "number",Symbol("data-bind") => "numericValue: value, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}"),:placeholder => "",:className => "input ",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}())], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
2, nothing),"value" => (Observable{Union{Nothing, Float64}} with 1 listeners. Value:
5.11, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "C:\\Users\\pshewale\\.julia\\packages\\Knockout\\1sDlc\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\pshewale\\.julia\\packages\\Knockout\\1sDlc\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\pshewale\\.julia\\packages\\InteractBase\\9mFwe\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\pshewale\\.julia\\packages\\InteractBase\\9mFwe\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\pshewale\\.julia\\packages\\Interact\\SbgIk\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"value" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"value\"]()) ? (this.valueFromJulia[\"value\"]=true, this.model[\"value\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[WebIO.IJuliaConnection(IJulia.CommManager.Comm{:webio_comm}("aa77f72f1f3048088494e1820098ca17", false, getfield(WebIO, Symbol("##92#93")){WebIO.IJuliaConnection}(WebIO.IJuliaConnection(#= circular reference @-3 =#)), IJulia.CommManager.noop_callback))]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(

The speed of the wave is: 0.7683284457478006 m/s
The wavelength of the wave is: 0.614192112138767 m
The frquency of this wave is: 1.2509578527022975 Hz
The amplitude of this wave is: 5.11 m
The phase constant of this wave is: 1.57 radians
